# ENI Port Scan

Use this runbook to diagnose the source of a port scan on an AWS ENI

## Parameters
* **eni_id**: The ENI ID of the network adapters receiving the port scan.
* **account_id**: THe AWS account that the port scan is occurring.

In [22]:
from jupyterirtools import sso

account_id = "251344881676"
eni_id = "eni-04050d5ac94cd243b"

sso.login("Jupyter-IR-ViewOnly", account_id)

If the windows doesn't automatically open, click on this https://device.sso.us-east-1.amazonaws.com/?user_code=SJDW-GLVZ to activate the session


<IPython.core.display.Javascript object>

In [24]:
from jupyterirtools import athena
from datetime import date
from datetime import timedelta

import importlib
importlib.reload(athena)

today = date.today()
yesterday = today - timedelta(days = 1)
    
sql = f"""SELECT srcaddr, srcport, dstaddr, dstport, count(packets) num_packets, sum(packets) packet_count, sum(bytes) sum_bytes
FROM "${{database_name}}"."vpc_flow_logs" 
WHERE "timestamp" >= '{str(yesterday.year).zfill(2)}/{str(yesterday.month).zfill(2)}/{str(yesterday.day).zfill(2)}'
AND accountid = '{account_id}' AND interface_id = '{eni_id}'
GROUP BY interface_id, srcaddr, srcport, dstaddr, dstport
ORDER BY sum(bytes) DESC
limit 1000;"""

#print(sql)
display(athena.run_query(sql))

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


profile_name=Jupyter-IR-ViewOnly-251344881676


,srcaddr,srcport,dstaddr,dstport,num_packets,packet_count,sum_bytes
0,52.216.251.86,443,172.31.89.172,48110,1,4392,6430942
1,52.217.135.224,443,172.31.89.172,49300,1,4386,6430633
2,52.216.56.74,443,172.31.89.172,42032,1,135,183376
3,52.46.156.29,443,172.31.89.172,53164,192,1986,129647
4,52.46.156.29,443,172.31.89.172,56298,161,1671,122759
...,...,...,...,...,...,...,...
995,52.46.156.217,443,172.31.89.172,47654,1,25,7249
996,67.220.242.18,443,172.31.89.172,54166,1,25,7249
997,52.46.148.67,443,172.31.89.172,49844,1,25,7249
998,67.220.240.170,443,172.31.89.172,44194,1,25,7249


# Recommended Actions
* Determine if IPs are internal or external.
* Perform a reputation lookup for the most frequently accessed external IPs. 
* Is a WAF rule needed to block the IP, or adjustments to the rate throttles.